### The Transport Problem

In [1]:
# Import the pyomo module
from pyomo.environ import *

# Creation of a Concrete Model
model = ConcreteModel()

In [2]:
# Define sets #

model.i = Set(initialize = ['seattle', 'san-diego'], doc = 'Canning plans')
model.j = Set(initialize = ['new-york', 'chicago', 'topeka'], doc = 'Markets')

In [3]:
# Define Parameters #

model.a  = Param(model.i, initialize = {'seattle':350, 'san-diego':600}, doc = 'Capacity of plant i in cases')
model.b  = Param(model.j, initialize = {'new-york':325, 'chicago':300, 'topeka':275}, doc = 'Demand in market j in cases')

dtab = {
    ('seattle', 'new-york'): 2.5,
    ('seattle', 'chicago'): 1.7,
    ('seattle', 'topeka'): 1.8,
    ('san-diego', 'new-york'): 2.5,
    ('san-diego', 'chicago') : 1.8,
    ('san-diego', 'topeka') : 1.4,
}

model.d = Param(model.i, model.j, initialize = dtab, doc = 'Distance in thousands of miles')

# Scalar f freight in dollars per case per thousand miles /90/ ;
model.f = Param(initialize = 90, doc = 'Freight in dollars per case per thousand miles')

In [4]:
# Parameter c(i, j) tranport cost in thousand of dollars per case ;

def c_init(model, i, j):
    return model.f * model.d[i, j] / 1000

model.c = Param(model.i, model.j, initialize = c_init, doc = 'Transport cost in thousands of dollars per case')

### Variables

In [5]:
## Define variables

model.x = Var(model.i, model.j, bounds = (0.0, None), doc = 'Shipment quantities in case')

### Constraints

In [6]:
## Define constrains 

def supply_rule(model, i):
    return sum(model.x[i, j] for j in model.j) <= model.a[i]
model.supply = Constraint(model.i, rule = supply_rule, doc = 'Observe supply limit at plant i')

def demand_rule(model, j):
    return sum(model.x[i, j] for i in model.i)  >= model.b[j]
model.demand = Constraint(model.j, rule = demand_rule, doc = 'Satisfy demand at market j')

### Objective and Solving

In [7]:
## Define objective and solve ##

def objective_rule(model):
    return sum(model.c[i, j]*model.x[i, j] for i in model.i for j in model.j)

model.objective = Objective(rule = objective_rule, sense = minimize, doc = 'Define objective function')

### Retrieving the Output

In [8]:
## Display the output

def pyomo_postprocess(options = None, instance = None, results = None):
    model.x.display()

In [9]:
model.pprint()

5 Set Declarations
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :    6 : {('seattle', 'new-york'), ('seattle', 'chicago'), ('seattle', 'topeka'), ('san-diego', 'new-york'), ('san-diego', 'chicago'), ('san-diego', 'topeka')}
    d_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :    6 : {('seattle', 'new-york'), ('seattle', 'chicago'), ('seattle', 'topeka'), ('san-diego', 'new-york'), ('san-diego', 'chicago'), ('san-diego', 'topeka')}
    i : Canning plans
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'seattle', 'san-diego'}
    j : Markets
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'new-york', 'chicago', 'topeka'}
    x_index : Size=1, Index=None, Ordered=True
     

### Editing and running script

In [10]:
## This is optional code path that allows the script to run outside of the pyomo command-line.

if __name__ == '__main__':
    # This emulates what the pyomo command-line tools does 
    
    from pyomo.opt import SolverFactory
    import pyomo.environ
    opt = SolverFactory('glpk')
    results = opt.solve(model)
    # sends results to stdout
    results.write()
    print('\nDisplaying Solution\n' + '-'*60)
    pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 153.675
  Upper bound: 153.675
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 13
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.02897024154663086
# ----------------------------------------------------------
#   Solution Information
# --------------------------------